[View in Colaboratory](https://colab.research.google.com/github/abunickabhi/IRproject/blob/master/Word_embedding.ipynb)

In [21]:
!pip3 install Cython
!pip3 install polyglot
!pip3 install word2vec
!pip3 install fasttext
!pip3 install gensim
!pip3 install torch

    13% |████▌                           | 69.4MB 48.7MB/s eta 0:00:09

    100% |████████████████████████████████| 496.4MB 2.5kB/s 


**Importing all the required libraries**

In [0]:
import gensim
import polyglot
import word2vec
import fasttext
import numpy as np
from gensim.models import fasttext
#from polyglot.text import Text, Word

**Word2Vec Training**



In [20]:
model = word2vec(size=100, window=5, min_count=5, workers=4)
model.save('/home/student/Documents/Dataset_Abhinav/english.vec')
model = word2vec.load('/home/student/Documents/Dataset_Abhinav/english.vec')


TypeError: ignored

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F

In [24]:
max_vocab = 100000
emb_dim = 300
src_lang = 'en'
src_emd_path = '/home/student/Documents/Dataset_Abhinav/english.vec'

src_word2id = {}
src_embeddings = []

with open(src_emd_path) as f:
    for i,line in enumerate(f):
        if i==0:
            split = line.split()
            assert len(split) == 2
            assert emb_dim == int(split[1])
        else:
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            if np.linalg.norm(vect)==0: #to avoid null embeddings
                vect[0] = 0.01
            assert word not in src_word2id
            assert vect.shape == (emb_dim, )
            src_word2id[word] = len(src_word2id)
            src_embeddings.append(vect[None,:])
        if i > max_vocab:
            break
            
src_id2word = {}
src_id2word = {v: k for k,v in src_word2id.items()}
src_embeddings = np.concatenate(src_embeddings,0)
src_embeddings = torch.from_numpy(src_embeddings).float()
src_embeddings = src_embeddings.cuda()
src_emb = nn.Embedding(len(src_word2id), emb_dim, sparse=True)
src_emb.weight.data.copy_(src_embeddings)

FileNotFoundError: ignored

In [25]:
from src.dictionary import Dictionary
src_dico = Dictionary(src_id2word, src_word2id, src_lang)
tgt_lang = 'ta'
tgt_emd_path = '/home/student/Documents/Dataset_Abhinav/tamil.vec'


tgt_word2id = {}
tgt_id2word = {}
tgt_embeddings = []



with open(tgt_emd_path) as f:
    for i,line in enumerate(f):
        if i==0:
            split = line.split()
            assert len(split) == 2
            assert emb_dim == int(split[1])
        else:
            word, vect = line.rstrip().split(' ', 1)
            vect = np.fromstring(vect, sep=' ')
            if np.linalg.norm(vect)==0: #to avoid null embeddings
                vect[0] = 0.01
            assert word not in tgt_word2id
            assert vect.shape == (emb_dim, )
            tgt_word2id[word] = len(tgt_word2id)
            tgt_embeddings.append(vect[None,:])
        if i > max_vocab:
            break

tgt_id2word = {v:k for k,v in tgt_word2id.items()}

tgt_dico = Dictionary(tgt_id2word, tgt_word2id, tgt_lang)
tgt_embeddings = np.concatenate(tgt_embeddings,0)
tgt_embeddings = torch.from_numpy(tgt_embeddings).float()
tgt_embeddings = tgt_embeddings.cuda()

tgt_emb = nn.Embedding(len(tgt_word2id), emb_dim, sparse=True)
tgt_emb.weight.data.copy_(tgt_embeddings)

mapping = nn.Linear(emb_dim, emb_dim, bias=False)
mapping.weight.data.copy_(torch.diag(torch.ones(emb_dim)))

disc_layers = 2
disc_dim_hidden = 2048
disc_dropout = 0
disc_inp_dropout = 0.1

ModuleNotFoundError: ignored

In [27]:
'''class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.emb_dim = emb_dim
        self.disc_layers = disc_layers
        self.disc_dim_hidden = disc_dim_hidden
        self.disc_dropout = disc_dropout
        self.disc_inp_dropout = disc_inp_dropout
        
        layers = [nn.Dropout(self.disc_inp_dropout)]
        for i in range(self.disc_layers + 1):
            input_dim = self.emb_dim if i == 0 else self.disc_dim_hidden
            output_dim = 1 if i==self.disc_layers else self.disc_dim_hidden
            layers.append(nn.Linear(input_dim, output_dim))
            if i < self.disc_layers:
                layers.append(nn.LeakyReLU(0.2))
                layers.append(nn.Dropout(self.disc_dropout))
        layers.append(nn.Sigmoid())
        self.layers = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.layers(x).view(-1)
      
discriminator = Discriminator()
src_emb.cuda()
tgt_emb.cuda()
mapping.cuda()
discriminator.cuda()'''

'class Discriminator(nn.Module):\n    def __init__(self):\n        super(Discriminator, self).__init__()\n        \n        self.emb_dim = emb_dim\n        self.disc_layers = disc_layers\n        self.disc_dim_hidden = disc_dim_hidden\n        self.disc_dropout = disc_dropout\n        self.disc_inp_dropout = disc_inp_dropout\n        \n        layers = [nn.Dropout(self.disc_inp_dropout)]\n        for i in range(self.disc_layers + 1):\n            input_dim = self.emb_dim if i == 0 else self.disc_dim_hidden\n            output_dim = 1 if i==self.disc_layers else self.disc_dim_hidden\n            layers.append(nn.Linear(input_dim, output_dim))\n            if i < self.disc_layers:\n                layers.append(nn.LeakyReLU(0.2))\n                layers.append(nn.Dropout(self.disc_dropout))\n        layers.append(nn.Sigmoid())\n        self.layers = nn.Sequential(*layers)\n        \n    def forward(self, x):\n        return self.layers(x).view(-1)\n      \ndiscriminator = Discriminator()